In [ ]:
from cosapp.drivers import MonteCarlo, RungeKutta, NonLinearSolver
from cosapp.recorders import DataFrameRecorder
from cosapp.utils.distributions import Normal

from cpu.systems import CPUSystem
from cpu.utils import createCPU

In [ ]:
cpu = CPUSystem("cpu")

In [ ]:
design = cpu.add_driver(NonLinearSolver('solver'))
design.extend(cpu.design_methods["exchanger_surface"])
design.runner.set_values({"fan.T_air": 40., "T_cpu": 80., "cpu.usage": 100.})

mc = cpu.add_driver(MonteCarlo('mc'))

time_driver = mc.add_child(RungeKutta(order=3, history=True))
time_driver.time_interval = (0, 30)
time_driver.dt = 0.5
# Define a simulation scenario
time_driver.set_scenario(
    init = {'T_cpu': 10.},
    values = {
        "cpu.usage": "100 if time <20 else 0."
    }
)

solver = time_driver.add_child(NonLinearSolver('solver', max_iter=10, factor=1.0))

mc.add_recorder(DataFrameRecorder(includes=['fan.tension', 'cpu.usage', 'T_cpu']))
mc.draws = 10

# parameters for uncertainties in the data
T_attr = cpu.fan.inwards.get_details('T_air')
# Set the distribution around the current value
T_attr.distribution = Normal(best=1, worst=-0.5)
mc.add_random_variable('fan.T_air')
mc.add_response(['fan.tension', 'cpu.usage', 'T_cpu'])

In [ ]:
cpu.run_drivers()
mc.recorder.export_data()